In [ ]:
import pandas as pd
import helpers_cbc
from helpers_eda_inicial import initial_eda
import numpy as np
import datetime
import time
import googlemaps
import calendar
import matplotlib.pyplot as plt

In [ ]:
CBC = '../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx'
# Load the data
xls = pd.ExcelFile(CBC)
for sheet_name in xls.sheet_names:
    print(sheet_name)

df_calificaciones = pd.read_excel(CBC, sheet_name='Calificaciones')
df_calificaciones = helpers_cbc.normalize_column_values(df_calificaciones, ['Carrera', 'Dirección', 'Localidad', 'dominio email', 'Materia', 'Nota', 'UBA XXI', 'Es materia FCEN?'])

In [ ]:
df_calificaciones

El objetivo de este notebook es probar la API de Google para poder calcular el tiempo de viaje de cada uno de los alumnos. El punto de partida sería la dirección en la columna Dirección.

In [ ]:
initial_eda(df_calificaciones)

Para la prueba que queremos hacer, lo que me interesa son las columnas direccion y localidad. Puedo quedarme con esas columnas junto con el DNI, luego puedo mergear ambos datasets en función del DNI.

In [ ]:
df_direcciones = pd.read_csv('../../assets/silver/enviados/datos_cbc_2020_2025.csv')
df_direcciones.columns

In [ ]:
df_direcciones = df_direcciones[['Dni', 'Dirección', 'Localidad']]

In [ ]:
df_direcciones.shape

La cantidad de datos faltantes disminuyó porque ahora cada alumno aparece una unica vez.

In [ ]:
df_direcciones.isnull().sum()

In [ ]:
df_direcciones['Dirección']

In [ ]:
df_direcciones['Localidad'].unique()

In [ ]:
df_direcciones[df_direcciones['Localidad']=='0']

Reemplazo los 0 por null

In [ ]:
df_direcciones['Localidad'] = df_direcciones['Localidad'].replace('0', np.nan)
df_direcciones['Dirección'] = df_direcciones['Dirección'].replace('0', np.nan)

In [ ]:
df_direcciones[df_direcciones['Localidad'].isnull()][~df_direcciones['Dirección'].isnull()]

In [ ]:
df_direcciones[df_direcciones['Dirección'].isnull()][~df_direcciones['Localidad'].isnull()]

In [ ]:
df_direcciones['Localidad'] = df_direcciones['Localidad'].replace('NO RESIDENTE', np.nan)

In [ ]:
initial_eda(df_direcciones)

## Probando la API

In [ ]:
with open('../../secrets/credential.txt', 'r') as file:
    API_KEY = file.read().strip()
gmaps = googlemaps.Client(key=API_KEY)

In [ ]:
origin = "Avenida Corrientes 123"
destination = "Intendente Güiraldes 2160, Ciudad Universitaria, Pabellón 2, Facultad de Ciencias Exactas y Naturales. Universidad de Buenos Aires. Argentina "

# Llamada al Distance Matrix API
result = gmaps.distance_matrix(origins=origin,
                               destinations=destination,
                               mode="transit", #le decimos que usamos el transporte publico
                               departure_time="now", # "now" para tiempo en tráfico
                               region = ".ar")  

# Extraer duración
element = result["rows"][0]["elements"][0]
duracion_texto = element["duration"]["text"]
duracion_segundos = element["duration"]["value"]
print(f"Tiempo de viaje: {duracion_texto} ({duracion_segundos} segundos)")

In [ ]:
result

In [ ]:
element

Para ciudad universitaria, pabellon 1, podemos usar el Place ID: ChIJRWvgB0i0vJURVpNj15qjgcM

Lo conseguimos en la pagina https://developers.google.com/maps/documentation/places/web-service/place-id?hl=es-419

In [ ]:
origin = "Avenida Corrientes 123"
place_id_destino = "ChIJRWvgB0i0vJURVpNj15qjgcM"

# Llamada al Distance Matrix API
result = gmaps.distance_matrix(origins=origin,
                               destinations=[f"place_id:{place_id_destino}"],
                               mode="transit", #le decimos que usamos el transporte publico
                               departure_time="now", # "now" para tiempo en tráfico
                               region = "AR")  

# Extraer duración
element = result["rows"][0]["elements"][0]
duracion_texto = element["duration"]["text"]
duracion_segundos = element["duration"]["value"]
print(f"Tiempo de viaje: {duracion_texto} ({duracion_segundos} segundos)")

In [ ]:
result

In [ ]:
geo = gmaps.geocode(f"place_id:ChIJMU0kiZ61vJURhkda1IVf9-0")
print(geo)

Probamos usando el Plus Code: FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires

Lo sacamos directamente de Google Maps.

In [ ]:
origin = "Bulnes - 1000, Palermo"
plus_code_destino = "FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires"

# Llamada al Distance Matrix API
result = gmaps.distance_matrix(origins=origin,
                               destinations=plus_code_destino,
                               mode="transit", #le decimos que usamos el transporte publico
                               departure_time="now", # "now" para tiempo en tráfico
                               region = "AR")  

# Extraer duración
element = result["rows"][0]["elements"][0]
duracion_texto = element["duration"]["text"]
duracion_segundos = element["duration"]["value"]
print(f"Tiempo de viaje: {duracion_texto} ({duracion_segundos} segundos)")

Vemos que ahora sí nos devuelve el lugar correcto. En los otros intentos reemplazaba el destination_addresses por otros más genericos. A su vez, notamos que es importante incluir la localidad en el origen, pues sin la aclaración de "Palermo" identificaba otra calle Guemes.

In [ ]:
result

## Con los datos de la FCEN

Me avisaron que nosotros vamos a tener código postal, localidad, departamento y provincia (y a su vez sabemos que hablamos de Argentina)

In [ ]:
codigo_postal= "C1425"
localidad= "Palermo"
departamento= "CABA"  # equivale a administrative_area_level_1
provincia= "Buenos Aires"

direccion = f"{codigo_postal}, {localidad}, {departamento}, {provincia}, Argentina"
resp = gmaps.geocode(direccion)

In [ ]:
resp

In [ ]:
loc = resp[0]["geometry"]["location"]

In [ ]:
loc

Me fije en google maps y estaba bastante bien. Veamos cómo le pasamos eso a distance_matrix

In [ ]:
lat_origen, lng_origen = loc["lat"], loc["lng"]
print(f"Origen en: {lat_origen}, {lng_origen}")

In [ ]:
# 2.1 Definir origen y destino
origins      = [(lat_origen, lng_origen)]            # lista de tuplas lat/lng
plus_code_destino = "FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires" # o place_id:<TU_ID> o plus code

# 2.2 Hacer la petición
result = gmaps.distance_matrix(
    origins=origins,
    destinations=plus_code_destino,
    mode="transit",                # modo: driving, walking, transit...
    departure_time="now", # 'now' para tráfico real en driving
    region="AR"                    # sesgo geográfico ISO-3166-1 alpha-2
)

In [ ]:
# Navegar la estructura de la respuesta
element = result["rows"][0]["elements"][0]

dist_text = element["distance"]["text"]   # ej. "10.1 km"
dist_val  = element["distance"]["value"]  # metros, ej. 10106

dur_text  = element["duration"]["text"]   # ej. "37 mins"
dur_val   = element["duration"]["value"]  # segundos, ej. 2240

print(f"Distancia: {dist_text} ({dist_val} m)")
print(f"Duración: {dur_text} ({dur_val} s)")


Probamos otro ejemplo:

En este caso el lugar queda a más de una hora y devuelve un texto distinto, entonces va a ser mejor trabajar con los segundos que devuelve y nosotros lo podemos transformar a minutos.

In [ ]:
#Conseguimos latitud y longitud de la dirección
codigo_postal= "C1407"
localidad= "Parque Avellaneda"
departamento= "CABA"  # equivale a administrative_area_level_1
provincia= "Buenos Aires"

direccion = f"{codigo_postal}, {localidad}, {departamento}, {provincia}, Argentina"
resp = gmaps.geocode(direccion)
loc = resp[0]["geometry"]["location"]
lat_origen, lng_origen = loc["lat"], loc["lng"]
print(f"Origen en: {lat_origen}, {lng_origen}")

# Conseguimos el tiempo de viaje
# 2.1 Definir origen y destino
origins      = [(lat_origen, lng_origen)]            # lista de tuplas lat/lng
plus_code_destino = "FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires" # o place_id:<TU_ID> o plus code

# 2.2 Hacer la petición
result = gmaps.distance_matrix(
    origins=origins,
    destinations=plus_code_destino,
    mode="transit",                # modo: driving, walking, transit...
    departure_time="now", # 'now' para tráfico real en driving
    region="AR"                    # sesgo geográfico ISO-3166-1 alpha-2
)
# Navegar la estructura de la respuesta
element = result["rows"][0]["elements"][0]

dist_text = element["distance"]["text"]   # ej. "10.1 km"
dist_val  = element["distance"]["value"]  # metros, ej. 10106

dur_text  = element["duration"]["text"]   # ej. "37 mins"
dur_val   = element["duration"]["value"]  # segundos, ej. 2240

print(f"Distancia: {dist_text} ({dist_val} m)")
print(f"Duración: {dur_text} ({dur_val} s)")

In [ ]:
codigo_postal= "6000"
localidad= "JUNIN"
departamento= "Junin"  # equivale a administrative_area_level_1
provincia= "Buenos Aires"

dia_llegada = datetime.datetime(2025, 5, 26, 9, 0)
arrival_time = calendar.timegm(dia_llegada.utctimetuple())

direccion = f"{localidad}, {codigo_postal}, {departamento}, {provincia}, Argentina"
plus_code_destino = "FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires" # o place_id:<TU_ID> o plus code
# 2.2 Hacer la petición
result = gmaps.distance_matrix(
    origins=direccion,
    destinations=plus_code_destino,
    mode="transit",                # modo: driving, walking, transit...
    departure_time=arrival_time, # 'now' para tráfico real en driving
    region="AR"                    # sesgo geográfico ISO-3166-1 alpha-2
)
# Navegar la estructura de la respuesta
element = result["rows"][0]["elements"][0]

dist_text = element["distance"]["text"]   # ej. "10.1 km"
dist_val  = element["distance"]["value"]  # metros, ej. 10106

dur_text  = element["duration"]["text"]   # ej. "37 mins"
dur_val   = element["duration"]["value"]  # segundos, ej. 2240

print(f"Distancia: {dist_text} ({dist_val} m)")
print(f"Duración: {dur_text} ({dur_val} s)")

In [ ]:
result

In [ ]:
arrival_time

Definimos una función con la que podríamos hacer el calculo:

In [ ]:
def distancia_de_viaje_cod_postal(localidad, codigo_postal, departamento, provincia, dia_llegada):
    print("entre")
    direccion = f"{codigo_postal}, {localidad}, {departamento}, {provincia}, Argentina"
    plus_code_destino = "FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires" # o place_id:<TU_ID> o plus code
    # 2.2 Hacer la petición
    result = gmaps.distance_matrix(
        origins=direccion,
        destinations=plus_code_destino,
        mode="transit",                # modo: driving, walking, transit...
        arrival_time=dia_llegada, # 'now' para tráfico real en driving
        region="AR"                    # sesgo geográfico ISO-3166-1 alpha-2
    )
    # Extraer duración
    element = result["rows"][0]["elements"][0]
    print(element)
    if element["status"] == "OK":
        duracion_texto = element["duration"]["text"]
        duracion_segundos = element["duration"]["value"]
        print(f"Tiempo de viaje: {duracion_texto} ({duracion_segundos} segundos)")
        return duracion_segundos
    else:
        return None  # o manejar errores según convenga

In [ ]:
personas = pd.read_csv('../../assets/silver/data_utilizada/personas_con_target.csv')
personas.head(5)

In [ ]:
personas.shape

In [ ]:
personas_datos_direccion = personas[['dni', 'localidad', 'codigo_postal', 'departamento', 'provincia', 'pais']]
personas_datos_direccion.head(5)

In [ ]:
personas_datos_direccion.drop_duplicates(inplace=True)

In [ ]:
personas_datos_direccion['codigo_postal'] = personas_datos_direccion['codigo_postal'].astype('Int64')

In [ ]:
personas_datos_direccion.head(5)

In [ ]:
# Nos quedamos con las filas donde todas las columnas excepto 'dni' son nulas
personas_datos_direccion = personas_datos_direccion[~personas_datos_direccion.drop(columns='dni').isna().all(axis=1)]

In [ ]:
personas_datos_direccion.shape

In [ ]:
personas_datos_direccion.isnull().sum()

In [ ]:
personas_datos_direccion['pais'].value_counts()

In [ ]:
personas_datos_direccion = personas_datos_direccion[personas_datos_direccion['pais'] == 'Argentina']

In [ ]:
personas_datos_direccion = personas_datos_direccion[~personas_datos_direccion['codigo_postal'].isnull()]

In [ ]:
personas_datos_direccion['codigo_postal'].unique()

In [ ]:
personas_datos_direccion.head(5)

In [ ]:
personas_datos_direccion.reset_index(drop=True, inplace=True)

In [ ]:
personas_datos_direccion_prueba = personas_datos_direccion[personas_datos_direccion['dni'] == '45617841'].copy()

In [ ]:
# No lo debería correr hasta que sepa bien qué valor poner en el horario de partida: departure_time
dia_llegada = datetime.datetime(2025, 5, 26, 17, 0)  # Martes 6 de mayo 2025, 9:00 am
arrival_time = int(time.mktime(dia_llegada.timetuple()))
for index, row in personas_datos_direccion.iterrows():
    localidad = row['localidad']
    codigo_postal = row['codigo_postal']
    departamento = row['departamento']
    provincia = row['provincia']
    duracion = distancia_de_viaje_cod_postal(localidad, codigo_postal, departamento, provincia, arrival_time)
    personas_datos_direccion.loc[index, 'duracion_tarde'] = duracion
    print(f"Vamos por el index {index}")
    print(f"Localidad: {localidad}, Codigo Postal: {codigo_postal}, Departamento: {departamento}, Provincia: {provincia} , Duración: {duracion}")

In [ ]:
personas_datos_direccion.head(5)

In [ ]:
personas_datos_direccion.to_csv('../../assets/silver/data_utilizada/datos_fcen_tiempo_viaje_tarde.csv', index=False)

In [ ]:
personas_datos_direccion.boxplot(column='duracion_manana')
plt.show()

In [ ]:
# haceme un histograma de la columna 'duracion_manana' de personas_datos_direccion
personas_datos_direccion.hist(column='duracion_manana', bins=50)
plt.xlabel('Duración (segundos)')
plt.ylabel('Frecuencia')
plt.title('Histograma de Duración del Viaje')
plt.show()

In [ ]:
# haceme un histograma de la columna 'duracion_manana' de personas_datos_direccion
personas_datos_direccion[personas_datos_direccion['duracion_manana'] < 14400].hist(column='duracion_manana', bins=50)
plt.xlabel('Duración (segundos)')
plt.ylabel('Frecuencia')
plt.title('Histograma de Duración del Viaje')
plt.show()

## Con los datos del CBC

In [ ]:
#Sacamos las filas que tienen valor nulo en ambas columnas
df_direcciones_sin_nulos = df_direcciones.dropna(subset=['Dirección', 'Localidad'], how='all')

Nos interesa probar de calcular el tiempo de viaje en los casos donde solo tenemos la dirección.

In [ ]:
df_direcciones_sin_nulos.isnull().sum()

Definimos una función que calcule la distancia de “{Dirección}, {Localidad}, Argentina” a Ciudad Universitaria.

In [ ]:
def distancia_de_viaje_direccion_y_localidad(direccion, localidad, horario_llegada, gmaps):
    origin = f"{direccion}, {localidad}, Argentina"
    plus_code_destino = "FH46+76 Buenos Aires, Cdad. Autónoma de Buenos Aires"

    # Llamada al Distance Matrix API
    result = gmaps.distance_matrix(origins=origin,
                                destinations=plus_code_destino,
                                mode="transit", #le decimos que usamos el transporte publico
                                arrival_time=horario_llegada, # "now" para tiempo en tráfico
                                region = "AR")  

    # Extraer duración
    element = result["rows"][0]["elements"][0]
    if element["status"] == "OK":
        duracion_texto = element["duration"]["text"]
        duracion_segundos = element["duration"]["value"]
        print(f"Tiempo de viaje: {duracion_texto} ({duracion_segundos} segundos)")
        return duracion_segundos
    else:
        return None  # o manejar errores según convenga

In [ ]:
# No lo debería correr hasta que sepa bien qué valor poner en el horario de partida: departure_time
dia_llegada = datetime.datetime(2025, 5, 6, 9, 0)  # Martes 6 de mayo 2025, 9:00 am
arrival_time = int(time.mktime(dia_llegada.timetuple()))
for index, row in df_direcciones_sin_nulos.iterrows():
    direccion = row['Dirección']
    localidad = row['Localidad']
    #duracion = distancia_de_viaje_direccion_y_localidad(direccion, localidad, arrival_time, gmaps) #lo comento por si corro la celda sin querer
    df_direcciones_sin_nulos.loc[index, 'Duracion_manana'] = duracion
    print(f"Vamos por el index {index}")
    print(f"Dirección: {direccion}, Localidad: {localidad}, Duración: {duracion}")

In [ ]:
#df_direcciones_sin_nulos.to_csv('../../assets/silver/distancia_viaje_cbc.csv', index=False)

In [ ]:
df_direcciones_sin_nulos = df_direcciones_sin_nulos.drop(columns=['Duracion'])

In [ ]:
#df_direcciones_sin_nulos.to_csv('../../assets/silver/distancia_viaje_cbc_solo_manana.csv', index=False)

In [ ]:
df_direcciones_sin_nulos.isnull().sum()

In [ ]:
df_direcciones_sin_nulos[df_direcciones_sin_nulos['Duracion_manana'].isnull()]

In [ ]:
df_direcciones_sin_nulos.boxplot(column='Duracion_manana')
plt.show()

Los que tardan 3 días es porque es un grupo de 8 personas que vivían en Ushuaia.

In [ ]:
df_direcciones_sin_nulos[df_direcciones_sin_nulos['Duracion_manana'] > 300000].shape

In [ ]:
df_direcciones_sin_nulos[df_direcciones_sin_nulos['Duracion_manana'] > 300000]

Las personas uqe tardan más de un día, vuelve a tener sentido porque viven fuera de Buenos Aires.

In [ ]:
df_direcciones_sin_nulos[(df_direcciones_sin_nulos['Duracion_manana'] > 86400) & (df_direcciones_sin_nulos['Duracion_manana'] < 300000)]

In [ ]:
df_direcciones_sin_nulos[(df_direcciones_sin_nulos['Duracion_manana'] > 43200) & (df_direcciones_sin_nulos['Duracion_manana'] < 86400)]['Localidad'].unique()

In [ ]:
df_direcciones_sin_nulos[(df_direcciones_sin_nulos['Duracion_manana'] > 43200) & (df_direcciones_sin_nulos['Duracion_manana'] < 86400)]

In [ ]:
df_direcciones_sin_nulos[(df_direcciones_sin_nulos['Duracion_manana'] > 14400) & (df_direcciones_sin_nulos['Duracion_manana'] < 43200)]['Localidad'].unique()

In [ ]:
df_direcciones_sin_nulos[df_direcciones_sin_nulos['Localidad'].isnull()]

Vemos que algunos de los casos donde devuelve null es porque realmente google maps no encuentra forma de llegar a ciudad universitaria utilizando el transporte público.

In [ ]:
df_direcciones_sin_nulos[df_direcciones_sin_nulos['Duracion_manana'].isnull()]